## 生成 tfRecord 文件
这个文件是在 win7 环境进行的工作。临时。
- 一般环境在 Ubuntu 虚拟机

In [1]:
import hashlib
import io
import logging
import os
import sys
import re
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util

d:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_dir = '../data/'
# group = 'train_b_1w'
# group = 'train_b_aug_1w'
group = 'train_b_1w_aug'
output_dir = '../data/new/'
label_map_path = '../../data/tinymind/race003-ehualu-object-detection/labels_items.txt'

In [3]:
def dict_to_tf_example(data, label_map_dict, image_subdirectory):
  filename, annotations = data
  img_path = os.path.join(image_subdirectory, filename)
  # print('img_path=', img_path)
  with tf.gfile.GFile(img_path, 'rb') as fid:
    encoded_jpg = fid.read()
  
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = PIL.Image.open(encoded_jpg_io)
#   print('image.format=', image.format)
  if image.format != 'JPEG':
    raise ValueError('Image format not JPEG')
  key = hashlib.sha256(encoded_jpg).hexdigest()
  width, height = image.size

  xmins = []
  ymins = []
  xmaxs = []
  ymaxs = []
  classes = []
  classes_text = []
  truncated = []
  poses = []
  difficult_obj = []

  annotation_list = annotations.split(';')
  for obj in annotation_list:
    difficult_obj.append(0)
    # x, y, width, height
    bbox = obj.split('_')
#     print('bbox=', bbox)
#     print('len(bbox)=', len(bbox))
    if not len(bbox) == 4:
        continue
    
    bbox = [int(float(bbox[0])), int(float(bbox[1])), int(float(bbox[2])), int(float(bbox[3]))]
    # print('bbox =', bbox)
    
    bbox[2] += bbox[0]
    bbox[3] += bbox[1]
    # xmin, ymin, xmax, ymax
    xmin, ymin, xmax, ymax = bbox

    xmins.append(xmin / width)
    ymins.append(ymin / height)
    xmaxs.append(xmax / width)
    ymaxs.append(ymax / height)

    classes_text.append('vehicle'.encode('utf8'))
    classes.append(1)
    truncated.append(0)
    poses.append('pose'.encode('utf8'))
    
  feature_dict = {
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
      'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
      'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
      'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
      'image/object/truncated': dataset_util.int64_list_feature(truncated),
      'image/object/view': dataset_util.bytes_list_feature(poses),
  }

  # print('feature_dict =', feature_dict)
  example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
  return example

In [4]:
def create_tf_record(output_filename, label_map_dict, image_dir, examples):
    writer = tf.python_io.TFRecordWriter(output_filename)
    total = examples.shape[0]
    for i in range(0, total):
        info = examples.iloc[i:i+1]
        # print('info =', info)
        name = info.name.values[0]
#         print('%s = %s' % (i, name))
        coordinate = info.coordinate.values[0]
#         print('coordinate =', coordinate)
        sys.stdout.write('\r>> On image %s/%s %s' % (i, total, name))
        sys.stdout.flush()
        # skip the rows contain no target  
        if len(coordinate) < 5:  
            print('%s==%s no coordinate' % (i,name))
            continue
#         try:
        tf_example = dict_to_tf_example([name, coordinate], label_map_dict, image_dir)
        writer.write(tf_example.SerializeToString())
#         except ValueError:
#             logging.warning('Invalid example: %s, ignoring.', name)
        # break
    writer.close()
    sys.stdout.write('\n')
    sys.stdout.flush()

In [5]:
# label_map_dict = label_map_util.get_label_map_dict(FLAGS.label_map_path)
label_map_dict = {}
logging.info('Reading from dataset.')
image_dir = os.path.join(data_dir, group)
annotations_path = os.path.join(data_dir, '%s.csv' % group)
annotations_info = pd.read_csv(annotations_path)

# 60000 * 0.01 = 60
train_examples, val_examples = train_test_split(annotations_info, test_size=0.01, random_state=42)
# print('num_examples=%s  num_train=%s' % (train_examples.shape, val_examples.shape))
logging.info('%s training and %s validation examples.', train_examples.shape, val_examples.shape)

train_output_path = os.path.join(output_dir, '%s_train.record'%group)
val_output_path = os.path.join(output_dir, '%s_val.record'%group)
  
create_tf_record(train_output_path, label_map_dict, image_dir, train_examples)
create_tf_record(val_output_path, label_map_dict, image_dir, val_examples)

>> On image 228/41387 e657f17c-5f07-4272-90db-75f17264117d.jpgb.jpg228==e657f17c-5f07-4272-90db-75f17264117d.jpg no coordinate
>> On image 729/41387 aug2_b9b78e5f-7edc-4747-952d-b5889983d7c3.jpg729==aug2_b9b78e5f-7edc-4747-952d-b5889983d7c3.jpg no coordinate
>> On image 6201/41387 aug2_963c2ab5-70a5-4dd4-af06-6ad3879fabb9.jpg6201==aug2_963c2ab5-70a5-4dd4-af06-6ad3879fabb9.jpg no coordinate
>> On image 8908/41387 e7298717-c6e7-4ea7-96cc-4d287908a956.jpge.jpg8908==e7298717-c6e7-4ea7-96cc-4d287908a956.jpg no coordinate
>> On image 9416/41387 f99c638b-f506-4841-b1b3-7e0fac27ce0a.jpg9.jpg9416==f99c638b-f506-4841-b1b3-7e0fac27ce0a.jpg no coordinate
>> On image 14049/41387 aug2_f99c638b-f506-4841-b1b3-7e0fac27ce0a.jpg14049==aug2_f99c638b-f506-4841-b1b3-7e0fac27ce0a.jpg no coordinate
>> On image 15566/41387 aug1_e657f17c-5f07-4272-90db-75f17264117d.jpg15566==aug1_e657f17c-5f07-4272-90db-75f17264117d.jpg no coordinate
>> On image 17011/41387 aug2_e7298717-c6e7-4ea7-96cc-4d287908a956.jpg17011==a